# ChatGPT Prompt Engineering for Developers

##  The Chat Format
utilizing the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

In [10]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [11]:
def get_completion(prompt, model = "gpt-3.5-turbo"):
    messages = [{"role":"user","content":prompt}]
    response = openai.ChatCompletion.create(
    model = model,
    messages = messages,
    temperature = 0)
    return response.choice[0].message["content"]

def get_completion_from_msg(messages,model = "gpt-3.5-turbo"):
    response = openai.ChatCompletion.create(
    model = model,
    messages = messages,
    temperature = 0)
    return response.choices[0].message["content"]

In [ ]:
messages = [
    {"role":"system", "content": "you are a friendly chatbot"},
    {"role":"user", "content": "Hi, my name is Markov"}]
    
response = get_completion_from_msg(messages)
response

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_msg(messages)
print(response)

Here the bot lost track and assumed it was asked the name of the user not the name assigned or given as input. 
We can fix this by **iteratively** improving our prompt by **Few shot learning**

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Markov'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_msg(messages)
print(response)

# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [ ]:
import panel as pn 

def collect_messages(_):
    prompt = inp.value_input
    inp.value = " "
    context.append({"role":"user","content":f"{prompt}"})
    response = get_completion_from_msg(context)
    context.append({"role":"assistant","content":f"{response}"})
    panels.append(
            pn.Row("User:", pn.pane.Markdown(prompt,width=600)))
    panels.append(
    pn.Row("Assistant",pn.pane.Markdown(response, width = 600)))
    
    return pn.Column(*panels)

In [ ]:
pn.extension()
panels = [] # to collect display 

context = [{"role":"system","content":"""
You are OrderBot, an automated service to collect orders for a Kerala restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
Biryani $50\
Coffee $10\
Chai $10\
Mutton Biriyani $70
Fries $5
Sauce $10
Coke $4
Pasta $7
Cheese Pasta $14"""}]

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=600),
)

dashboard
